<div align=center><h1>Individual Project PIº1</h1>
<h4>Salomón Orozco Jaramillo</h4>
<h4>ETL (Extract, Transform, Load)
</h4>
</div>

## Importing Libs

In [2]:
# Importing necessary libraries
import ast  # For working with Python literal syntax
import json  # For working with JSON files
import pandas as pd  # For data manipulation
import nltk  # Natural Language Toolkit
import re # Regular Expression library
import nltk # Use the nltk library with vader_lexicon and sentiment to transform reviews into sentiment scores
nltk.download('vader_lexicon') # Download  VADER lexicon from NLTK
from nltk.sentiment.vader import SentimentIntensityAnalyzer # Initiate  the sentiment analyzer
import seaborn
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## australian_users_items.json etl

In [3]:
# Creating a list for the items
review = []
# Opening json
with open('datasets/australian_users_items.json', 'r', encoding='utf-8') as usersItemJson:
    for line in usersItemJson:
        # Append json items to the list
        review.append(ast.literal_eval(line))
# Close the json file explicitly
usersItemJson.close()
# Creating a dataframe
df_user_items = pd.DataFrame(review)
# Viewing the dataframe
df_user_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


### Desempaquetar la columna 'Items'

In [4]:
# Unpacking the 'Items' column
data_user_temp1 = df_user_items.explode(['items'])
# Normalizing JSON data in the 'items' column
data_user_temp2 = pd.json_normalize(data_user_temp1['items']).set_index(data_user_temp1['items'].index)
# Concatenating normalized data with the original DataFrame
data_user_temp3 = pd.concat([data_user_temp2, data_user_temp1], axis=1)
# Updating the original DataFrame with the concatenated data
df_user_items = data_user_temp3

In [5]:
# Remove the 'items' column from the DataFrame
df_user_items = df_user_items.drop('items', axis=1)

In [6]:
# View DataFrame
df_user_items

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
0,10,Counter-Strike,6.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
0,20,Team Fortress Classic,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
0,30,Day of Defeat,7.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
0,40,Deathmatch Classic,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
0,50,Half-Life: Opposing Force,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
88308,373330,All Is Dust,0.0,0.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
88308,388490,One Way To Die: Steam Edition,3.0,3.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
88308,521570,You Have 10 Seconds 2,4.0,4.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
88308,519140,Minds Eyes,3.0,3.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


In [7]:
# Check the 'item_id' content data for a specific user ('76561198329548331')
df_user_items.loc[df_user_items['user_id'] == '76561198329548331']

#df_user_items['steam_id'].info()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
88308,304930,Unturned,677.0,677.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
88308,227940,Heroes & Generals,43.0,43.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
88308,346330,BrainBread 2,0.0,0.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
88308,373330,All Is Dust,0.0,0.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
88308,388490,One Way To Die: Steam Edition,3.0,3.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
88308,521570,You Have 10 Seconds 2,4.0,4.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
88308,519140,Minds Eyes,3.0,3.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


In [8]:
# Calculate the number of null values per column in df_user_items
null_per_col = df_user_items.isnull().sum()

#null_per_col

In [9]:
# Remove columns that are not needed
df_user_items = df_user_items.drop('user_url', axis=1)
df_user_items = df_user_items.drop('items_count', axis=1)

In [10]:
# View DataFrame

df_user_items

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,steam_id
0,10,Counter-Strike,6.0,0.0,76561197970982479,76561197970982479
0,20,Team Fortress Classic,0.0,0.0,76561197970982479,76561197970982479
0,30,Day of Defeat,7.0,0.0,76561197970982479,76561197970982479
0,40,Deathmatch Classic,0.0,0.0,76561197970982479,76561197970982479
0,50,Half-Life: Opposing Force,0.0,0.0,76561197970982479,76561197970982479
...,...,...,...,...,...,...
88308,373330,All Is Dust,0.0,0.0,76561198329548331,76561198329548331
88308,388490,One Way To Die: Steam Edition,3.0,3.0,76561198329548331,76561198329548331
88308,521570,You Have 10 Seconds 2,4.0,4.0,76561198329548331,76561198329548331
88308,519140,Minds Eyes,3.0,3.0,76561198329548331,76561198329548331


In [11]:
# Display information about df_user_items
df_user_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5170015 entries, 0 to 88309
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           object 
 1   item_name         object 
 2   playtime_forever  float64
 3   playtime_2weeks   float64
 4   user_id           object 
 5   steam_id          object 
dtypes: float64(2), object(4)
memory usage: 276.1+ MB


In [12]:
# Find NaN values and drop corresponding rows for specific columns
df_user_items = df_user_items.dropna(subset=['item_id'])
df_user_items = df_user_items.dropna(subset=['item_name'])
df_user_items = df_user_items.dropna(subset=['playtime_forever'])
df_user_items = df_user_items.dropna(subset=['playtime_2weeks'])
df_user_items = df_user_items.dropna(subset=['steam_id'])
df_user_items = df_user_items.dropna(subset=['user_id'])

In [13]:
# Dictionary mapping old column names to new column names
column_mapping = {
    'user_id': 'User_Id',
    'item_id': 'Item_Id',
    'playtime_forever': 'Playtime_Forever',
    'item_name': 'Item_Name',
    'steam_id': 'Steam_Id',
    'playtime_2weeks': 'Playtime_2Weeks'
}

# Rename columns using the dictionary
df_user_items = df_user_items.rename(columns=column_mapping)

In [14]:
# View Dataframe
df_user_items

,Item_Id,Item_Name,Playtime_Forever,Playtime_2Weeks,User_Id,Steam_Id
0,10,Counter-Strike,6.0,0.0,76561197970982479,76561197970982479
0,20,Team Fortress Classic,0.0,0.0,76561197970982479,76561197970982479
0,30,Day of Defeat,7.0,0.0,76561197970982479,76561197970982479
0,40,Deathmatch Classic,0.0,0.0,76561197970982479,76561197970982479
0,50,Half-Life: Opposing Force,0.0,0.0,76561197970982479,76561197970982479
...,...,...,...,...,...,...
88308,346330,BrainBread 2,0.0,0.0,76561198329548331,76561198329548331
88308,373330,All Is Dust,0.0,0.0,76561198329548331,76561198329548331
88308,388490,One Way To Die: Steam Edition,3.0,3.0,76561198329548331,76561198329548331
88308,521570,You Have 10 Seconds 2,4.0,4.0,76561198329548331,76561198329548331


In [15]:
# Get DataFrame info
df_user_items.info()
duplicates = df_user_items.duplicated()
print(duplicates.sum())

<class 'pandas.core.frame.DataFrame'>
Index: 5153209 entries, 0 to 88308
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Item_Id           object 
 1   Item_Name         object 
 2   Playtime_Forever  float64
 3   Playtime_2Weeks   float64
 4   User_Id           object 
 5   Steam_Id          object 
dtypes: float64(2), object(4)
memory usage: 275.2+ MB
59104


### Export Data Files

In [16]:
# Save the DataFrame to a CSV file
# df_user_items.to_csv('CSV/user_items.csv', index=False)


In [17]:
# Save the DataFrame to a compressed Parquet file
df_user_items.to_parquet('GZIP/user_items.gzip', compression='gzip')

## output_steam_games.json etl

In [18]:
# List to store JSON objects
games = []
# Read 'output_steam_games.json' line by line and load as JSON
with open('datasets/output_steam_games.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        games.append(json.loads(line))
# Create a DataFrame from the list of JSON objects
df_steam_games = pd.DataFrame(games)

In [19]:
# View DataFrame
df_steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [20]:
# Dictionary mapping old column names to new column names
column_mapping = {
    'publisher': 'Publisher',
    'genres': 'Genres',
    'app_name': 'App_Name',
    'title': 'Title',
    'url': 'Url',
    'release_date': 'Release_Date',
    'tags': 'Tags',
    'reviews_url': 'Reviews_Url',
    'specs': 'Specs',
    'price': 'Price',
    'early_access': 'Early_Access',
    'id': 'Id',
    'developer': 'Developer'
}
# Rename columns using the dictionary
df_steam_games = df_steam_games.rename(columns=column_mapping)

In [21]:
# View new column names
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Publisher     24083 non-null  object
 1   Genres        28852 non-null  object
 2   App_Name      32133 non-null  object
 3   Title         30085 non-null  object
 4   Url           32135 non-null  object
 5   Release_Date  30068 non-null  object
 6   Tags          31972 non-null  object
 7   Reviews_Url   32133 non-null  object
 8   Specs         31465 non-null  object
 9   Price         30758 non-null  object
 10  Early_Access  32135 non-null  object
 11  Id            32133 non-null  object
 12  Developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [22]:
def normalize_col(col):
    """
    Normalize a developer name by removing non-alphabetic characters,
    converting to lowercase, and removing extra whitespaces.
    """
    if isinstance(col, str):  # Verify if it is a string
        cleaned_name = re.sub(r'[^a-zA-Z\s]', '', col)
        new_name = re.sub(r'\s+', ' ', cleaned_name.lower().strip())
        return new_name
    else:
        return col

# Apply the normalize_col function to the 'Publisher' column
df_steam_games['Publisher'] = df_steam_games['Publisher'].apply(normalize_col)


In [23]:
# Remove the 'Url' column
df_steam_games.drop('Url', axis=1, inplace=True)

In [24]:
df_steam_games

,Publisher,Genres,App_Name,Title,Release_Date,Tags,Reviews_Url,Specs,Price,Early_Access,Id,Developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
120440,ghostrus games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,laush studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,sixnails,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [25]:
# Function to convert float to string
def float_to_string(value):
    """
    Convert a float to string. If the value is not a float, return it unchanged.
    """
    if isinstance(value, float):
        return str(value)
    return value
# Apply the float_to_str function to the 'Publisher' column
df_steam_games['Publisher'] = df_steam_games['Publisher'].apply(float_to_string)


In [26]:
# Capitalize the first letter of each word in the 'Publisher' column
df_steam_games['Publisher'] = df_steam_games['Publisher'].str.title()

In [27]:
# Count Null values in 'Publisher'
nullPublisher = df_steam_games['Publisher'].isnull().sum()
print("Null values in 'Publisher':",nullPublisher)

# Count NaN values in 'Publisher'
nans_as_strings = (df_steam_games['Publisher'].str.lower() == 'nan').sum()
# Print the number of NaN values represented as strings in 'Publisher'
print("NaN values in 'Publisher':", nans_as_strings)
print('--------')
# Count Null values in 'Publisher'
nullPublisher = df_steam_games['Genres'].isnull().sum()
print("Null values in 'Genres':",nullPublisher)

Null values in 'Publisher': 0
NaN values in 'Publisher': 96362
--------
Null values in 'Genres': 91593


In [28]:
# Check Genres content
print(df_steam_games['Genres'])

0                                           NaN
1                                           NaN
2                                           NaN
3                                           NaN
4                                           NaN
                          ...                  
120440    [Casual, Indie, Simulation, Strategy]
120441                [Casual, Indie, Strategy]
120442              [Indie, Racing, Simulation]
120443                          [Casual, Indie]
120444                                      NaN
Name: Genres, Length: 120445, dtype: object


In [29]:
# Convert the values in the 'Genres' column to a comma-separated string
df_steam_games['Genres'] = df_steam_games['Genres'].astype(str).apply(lambda x: ', '.join(filter(None, x.split(','))))

In [30]:
df_steam_games

,Publisher,Genres,App_Name,Title,Release_Date,Tags,Reviews_Url,Specs,Price,Early_Access,Id,Developer
0,Nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghostrus Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,Russian Roads,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,Sixnails,"['Casual', 'Indie']",EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [31]:
# Remove rows with NaN values in 'Genres'
df_steam_games.dropna(subset=['Genres'], inplace=True)

In [32]:
df_steam_games['Genres']

0                                                      nan
1                                                      nan
2                                                      nan
3                                                      nan
4                                                      nan
                                ...                       
120440    ['Casual',  'Indie',  'Simulation',  'Strategy']
120441                   ['Casual',  'Indie',  'Strategy']
120442                 ['Indie',  'Racing',  'Simulation']
120443                                ['Casual',  'Indie']
120444                                                 nan
Name: Genres, Length: 120445, dtype: object

In [33]:
df_steam_games['Specs']

0                                                       NaN
1                                                       NaN
2                                                       NaN
3                                                       NaN
4                                                       NaN
                                ...                        
120440                  [Single-player, Steam Achievements]
120441    [Single-player, Steam Achievements, Steam Clou...
120442    [Single-player, Steam Achievements, Steam Trad...
120443     [Single-player, Steam Achievements, Steam Cloud]
120444    [Single-player, Stats, Steam Leaderboards, HTC...
Name: Specs, Length: 120445, dtype: object

In [34]:
# Count null values in 'Specs'
nullSpecs = df_steam_games['Specs'].isnull().sum()
# Print the number of null values in 'Specs'
print("Null values in 'Specs':", nullSpecs)


Null values in 'Specs': 88980


In [35]:
# Remove rows with NaN values in 'Specs'
df_steam_games.dropna(subset=['Specs'], inplace=True)

In [36]:
# Extract 'Specs' specifications
df_steam_games['Specs'] = df_steam_games['Specs'].apply(lambda x: ', '.join(filter(None, x)))


In [37]:
df_steam_games

,Publisher,Genres,App_Name,Title,Release_Date,Tags,Reviews_Url,Specs,Price,Early_Access,Id,Developer
88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation',...",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,Single-player,4.99,False,761140,Kotoshiro
88311,Making Fun Inc,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"Single-player, Multi-player, Online Multi-Play...",Free To Play,False,643980,Secret Level SRL
88312,Poolianscom,"['Casual', 'Free to Play', 'Indie', 'Simula...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"Single-player, Multi-player, Online Multi-Play...",Free to Play,False,670290,Poolians.com
88313,,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,Single-player,0.99,False,767400,彼岸领域
88314,Nan,nan,Log Challenge,NaN,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"Single-player, Full controller support, HTC Vi...",2.99,False,773570,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghostrus Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"Single-player, Steam Achievements",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"Single-player, Steam Achievements, Steam Cloud...",4.99,False,733530,Sacada
120442,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,Russian Roads,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"Single-player, Steam Achievements, Steam Tradi...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,Sixnails,"['Casual', 'Indie']",EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"Single-player, Steam Achievements, Steam Cloud",4.99,False,658870,"xropi,stev3ns"


In [38]:
# Count null values in 'title'
null_count = df_steam_games['Title'].isnull().sum()
# Print the number of null values in 'title'
print(f"Null values in 'Title': {null_count}")
# Count null app_name in 'App_name'
null_count = df_steam_games['App_Name'].isnull().sum()
# Print the number of null values in 'title'
print(f"Null values in 'App_Name': {null_count}")

Null values in 'Title': 2049
Null values in 'App_Name': 1


In [39]:
# Drop Title column
df_steam_games = df_steam_games.drop('Title', axis=1)

In [40]:
# remove Na in App_Name
df_steam_games.dropna(subset=['App_Name'], inplace=True)

In [41]:
df_steam_games

,Publisher,Genres,App_Name,Release_Date,Tags,Reviews_Url,Specs,Price,Early_Access,Id,Developer
88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation',...",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,Single-player,4.99,False,761140,Kotoshiro
88311,Making Fun Inc,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"Single-player, Multi-player, Online Multi-Play...",Free To Play,False,643980,Secret Level SRL
88312,Poolianscom,"['Casual', 'Free to Play', 'Indie', 'Simula...",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"Single-player, Multi-player, Online Multi-Play...",Free to Play,False,670290,Poolians.com
88313,,"['Action', 'Adventure', 'Casual']",弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,Single-player,0.99,False,767400,彼岸领域
88314,Nan,nan,Log Challenge,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"Single-player, Full controller support, HTC Vi...",2.99,False,773570,NaN
...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghostrus Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"Single-player, Steam Achievements",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"Single-player, Steam Achievements, Steam Cloud...",4.99,False,733530,Sacada
120442,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"Single-player, Steam Achievements, Steam Tradi...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,Sixnails,"['Casual', 'Indie']",EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"Single-player, Steam Achievements, Steam Cloud",4.99,False,658870,"xropi,stev3ns"


In [42]:
# Normalize App_Name column names
df_steam_games['App_Name'] = df_steam_games['App_Name'].apply(normalize_col)

In [43]:
# Capitalize App name column
df_steam_games['App_Name'] = df_steam_games['App_Name'].str.title()

In [44]:
# Remove columns 'reviews_url' and 'tags'
remove_columns = ['Reviews_Url', 'Tags']
df_steam_games = df_steam_games.drop(columns=remove_columns)


In [45]:
# View DataFrame
df_steam_games

,Publisher,Genres,App_Name,Release_Date,Specs,Price,Early_Access,Id,Developer
88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation',...",Lost Summoner Kitty,2018-01-04,Single-player,4.99,False,761140,Kotoshiro
88311,Making Fun Inc,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,"Single-player, Multi-player, Online Multi-Play...",Free To Play,False,643980,Secret Level SRL
88312,Poolianscom,"['Casual', 'Free to Play', 'Indie', 'Simula...",Real Pool D Poolians,2017-07-24,"Single-player, Multi-player, Online Multi-Play...",Free to Play,False,670290,Poolians.com
88313,,"['Action', 'Adventure', 'Casual']",,2017-12-07,Single-player,0.99,False,767400,彼岸领域
88314,Nan,nan,Log Challenge,NaN,"Single-player, Full controller support, HTC Vi...",2.99,False,773570,NaN
...,...,...,...,...,...,...,...,...,...
120440,Ghostrus Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,2018-01-04,"Single-player, Steam Achievements",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"['Casual', 'Indie', 'Strategy']",Logistical South Africa,2018-01-04,"Single-player, Steam Achievements, Steam Cloud...",4.99,False,733530,Sacada
120442,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,2018-01-04,"Single-player, Steam Achievements, Steam Tradi...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,Sixnails,"['Casual', 'Indie']",Exit Directions,2017-09-02,"Single-player, Steam Achievements, Steam Cloud",4.99,False,658870,"xropi,stev3ns"


In [46]:
# Convert 'Release_Date' to datetime objects
df_steam_games['Release_Date'] = pd.to_datetime(df_steam_games['Release_Date'], dayfirst=True, errors='coerce')

In [47]:
# Null columns in 'Release_Date'
null_release = df_steam_games['Release_Date'].isnull().sum()
print(null_release)

20314


In [48]:
# remove na from Release_Date
df_steam_games.dropna(subset = ['Release_Date'], inplace=True)

In [49]:
df_steam_games

,Publisher,Genres,App_Name,Release_Date,Specs,Price,Early_Access,Id,Developer
88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation',...",Lost Summoner Kitty,2018-04-01,Single-player,4.99,False,761140,Kotoshiro
88311,Making Fun Inc,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-04-01,"Single-player, Multi-player, Online Multi-Play...",Free To Play,False,643980,Secret Level SRL
88313,,"['Action', 'Adventure', 'Casual']",,2017-07-12,Single-player,0.99,False,767400,彼岸领域
88315,Trickjump Games Ltd,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,2018-04-01,"Single-player, Steam Achievements",3.99,False,772540,Trickjump Games Ltd
88316,Nan,"['Free to Play', 'Indie', 'Simulation', 'Sp...",Snow All Access Basic Pass,2018-04-01,"Single-player, Multi-player, Online Multi-Play...",9.99,False,774276,Poppermost Productions
...,...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"['Action', 'Adventure', 'Casual', 'Indie']",Kebab It Up,2018-04-01,"Single-player, Steam Achievements, Steam Cloud",1.99,False,745400,Bidoniera Games
120440,Ghostrus Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,2018-04-01,"Single-player, Steam Achievements",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"['Casual', 'Indie', 'Strategy']",Logistical South Africa,2018-04-01,"Single-player, Steam Achievements, Steam Cloud...",4.99,False,733530,Sacada
120442,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,2018-04-01,"Single-player, Steam Achievements, Steam Tradi...",1.99,False,610660,Laush Dmitriy Sergeevich


In [50]:
# Add column 'Release_Year' with the year from 'Release_Date'
df_steam_games['Release_Year'] = pd.to_datetime(df_steam_games['Release_Date']).dt.year

In [51]:
# View DataFrame
df_steam_games

,Publisher,Genres,App_Name,Release_Date,Specs,Price,Early_Access,Id,Developer,Release_Year
88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation',...",Lost Summoner Kitty,2018-04-01,Single-player,4.99,False,761140,Kotoshiro,2018
88311,Making Fun Inc,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-04-01,"Single-player, Multi-player, Online Multi-Play...",Free To Play,False,643980,Secret Level SRL,2018
88313,,"['Action', 'Adventure', 'Casual']",,2017-07-12,Single-player,0.99,False,767400,彼岸领域,2017
88315,Trickjump Games Ltd,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,2018-04-01,"Single-player, Steam Achievements",3.99,False,772540,Trickjump Games Ltd,2018
88316,Nan,"['Free to Play', 'Indie', 'Simulation', 'Sp...",Snow All Access Basic Pass,2018-04-01,"Single-player, Multi-player, Online Multi-Play...",9.99,False,774276,Poppermost Productions,2018
...,...,...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"['Action', 'Adventure', 'Casual', 'Indie']",Kebab It Up,2018-04-01,"Single-player, Steam Achievements, Steam Cloud",1.99,False,745400,Bidoniera Games,2018
120440,Ghostrus Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,2018-04-01,"Single-player, Steam Achievements",1.99,False,773640,"Nikita ""Ghost_RUS""",2018
120441,Sacada,"['Casual', 'Indie', 'Strategy']",Logistical South Africa,2018-04-01,"Single-player, Steam Achievements, Steam Cloud...",4.99,False,733530,Sacada,2018
120442,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,2018-04-01,"Single-player, Steam Achievements, Steam Tradi...",1.99,False,610660,Laush Dmitriy Sergeevich,2018


In [52]:
# apply float_to_string function to 'Genres' column
df_steam_games['Genres'] = df_steam_games['Genres'].apply(float_to_string)

In [53]:
df_steam_games

,Publisher,Genres,App_Name,Release_Date,Specs,Price,Early_Access,Id,Developer,Release_Year
88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation',...",Lost Summoner Kitty,2018-04-01,Single-player,4.99,False,761140,Kotoshiro,2018
88311,Making Fun Inc,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-04-01,"Single-player, Multi-player, Online Multi-Play...",Free To Play,False,643980,Secret Level SRL,2018
88313,,"['Action', 'Adventure', 'Casual']",,2017-07-12,Single-player,0.99,False,767400,彼岸领域,2017
88315,Trickjump Games Ltd,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,2018-04-01,"Single-player, Steam Achievements",3.99,False,772540,Trickjump Games Ltd,2018
88316,Nan,"['Free to Play', 'Indie', 'Simulation', 'Sp...",Snow All Access Basic Pass,2018-04-01,"Single-player, Multi-player, Online Multi-Play...",9.99,False,774276,Poppermost Productions,2018
...,...,...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"['Action', 'Adventure', 'Casual', 'Indie']",Kebab It Up,2018-04-01,"Single-player, Steam Achievements, Steam Cloud",1.99,False,745400,Bidoniera Games,2018
120440,Ghostrus Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,2018-04-01,"Single-player, Steam Achievements",1.99,False,773640,"Nikita ""Ghost_RUS""",2018
120441,Sacada,"['Casual', 'Indie', 'Strategy']",Logistical South Africa,2018-04-01,"Single-player, Steam Achievements, Steam Cloud...",4.99,False,733530,Sacada,2018
120442,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,2018-04-01,"Single-player, Steam Achievements, Steam Tradi...",1.99,False,610660,Laush Dmitriy Sergeevich,2018


In [54]:
# Function to check if a value is a string and convert to 0
def string_to_zero(value):
    if isinstance(value, str):
        return 0
    else:
        return value

In [55]:
# Check na in 'Price' column
df_steam_games['Price'].isna().sum()

375

In [56]:
# Delete na rows in 'Price' column
df_steam_games.dropna(subset= ['Price'], inplace=True)

In [57]:
# Apply the string_to_zero function in the free to play column
df_steam_games['Price'] = df_steam_games['Price'].apply(string_to_zero)

In [58]:
# Check the number of non-numeric rows in the 'Price' column
non_numeric_count = pd.to_numeric(df_steam_games['Price'], errors='coerce').isnull().sum()
print(f"Number of rows with non-numeric values in 'Price': {non_numeric_count}")

Number of rows with non-numeric values in 'Price': 0


In [59]:
# Convert 'Price' column to float type
df_steam_games['Price'] = df_steam_games['Price'].astype(float)


In [60]:
df_steam_games['Early_Access'].info()

<class 'pandas.core.series.Series'>
Index: 10775 entries, 88310 to 120443
Series name: Early_Access
Non-Null Count  Dtype 
--------------  ----- 
10775 non-null  object
dtypes: object(1)
memory usage: 168.4+ KB


In [61]:
# Convert objet to bool
df_steam_games['Early_Access'] = df_steam_games['Early_Access'].astype(bool)

In [62]:
# Count null values in 'Early_Access'
null_count = df_steam_games['Early_Access'].isnull().sum()
# Print the number of null values in 'Early_Access' column
print("\nNumber of null values in 'Early_Access' column:", null_count)


Number of null values in 'Early_Access' column: 0


In [63]:
# Check Developer names
df_steam_games

,Publisher,Genres,App_Name,Release_Date,Specs,Price,Early_Access,Id,Developer,Release_Year
88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation',...",Lost Summoner Kitty,2018-04-01,Single-player,4.99,False,761140,Kotoshiro,2018
88311,Making Fun Inc,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-04-01,"Single-player, Multi-player, Online Multi-Play...",0.00,False,643980,Secret Level SRL,2018
88313,,"['Action', 'Adventure', 'Casual']",,2017-07-12,Single-player,0.99,False,767400,彼岸领域,2017
88315,Trickjump Games Ltd,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,2018-04-01,"Single-player, Steam Achievements",3.99,False,772540,Trickjump Games Ltd,2018
88316,Nan,"['Free to Play', 'Indie', 'Simulation', 'Sp...",Snow All Access Basic Pass,2018-04-01,"Single-player, Multi-player, Online Multi-Play...",9.99,False,774276,Poppermost Productions,2018
...,...,...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"['Action', 'Adventure', 'Casual', 'Indie']",Kebab It Up,2018-04-01,"Single-player, Steam Achievements, Steam Cloud",1.99,False,745400,Bidoniera Games,2018
120440,Ghostrus Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,2018-04-01,"Single-player, Steam Achievements",1.99,False,773640,"Nikita ""Ghost_RUS""",2018
120441,Sacada,"['Casual', 'Indie', 'Strategy']",Logistical South Africa,2018-04-01,"Single-player, Steam Achievements, Steam Cloud...",4.99,False,733530,Sacada,2018
120442,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,2018-04-01,"Single-player, Steam Achievements, Steam Tradi...",1.99,False,610660,Laush Dmitriy Sergeevich,2018


In [64]:
# Normalize 'Developer' names
df_steam_games['Developer'] = df_steam_games['Developer'].apply(normalize_col)

In [65]:
# Capitalize the first letter of each word in 'Developer' column
df_steam_games['Developer'] = df_steam_games['Developer'].str.title()

In [66]:
# Count null values in 'Developer' column
null_count = df_steam_games['Developer'].isnull().sum()
# Print the number of null values in 'Developer' column
print("\nNumber of null values in 'Developer' column:", null_count)


Number of null values in 'Developer' column: 400


In [67]:
# Count null values in 'Developer' column
null_count = df_steam_games['Id'].isnull().sum()
# Print the number of null values in 'Developer' column
print("\nNumber of null values in 'Id' column:", null_count)


Number of null values in 'Id' column: 1


In [68]:
# Drop rows with null values in 'Developer' column
df_steam_games.dropna(subset=['Developer'], inplace=True)

In [69]:
# Drop rows with null values in 'Developer' column
df_steam_games.dropna(subset=['Id'], inplace=True)

In [70]:
# Count null values in 'Developer' column
null_count = df_steam_games['Id'].isnull().sum()
# Print the number of null values in 'Developer' column
print("\nNumber of null values in 'id' column:", null_count)


Number of null values in 'id' column: 0


In [71]:
df_steam_games

,Publisher,Genres,App_Name,Release_Date,Specs,Price,Early_Access,Id,Developer,Release_Year
88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation',...",Lost Summoner Kitty,2018-04-01,Single-player,4.99,False,761140,Kotoshiro,2018
88311,Making Fun Inc,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-04-01,"Single-player, Multi-player, Online Multi-Play...",0.00,False,643980,Secret Level Srl,2018
88313,,"['Action', 'Adventure', 'Casual']",,2017-07-12,Single-player,0.99,False,767400,,2017
88315,Trickjump Games Ltd,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,2018-04-01,"Single-player, Steam Achievements",3.99,False,772540,Trickjump Games Ltd,2018
88316,Nan,"['Free to Play', 'Indie', 'Simulation', 'Sp...",Snow All Access Basic Pass,2018-04-01,"Single-player, Multi-player, Online Multi-Play...",9.99,False,774276,Poppermost Productions,2018
...,...,...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"['Action', 'Adventure', 'Casual', 'Indie']",Kebab It Up,2018-04-01,"Single-player, Steam Achievements, Steam Cloud",1.99,False,745400,Bidoniera Games,2018
120440,Ghostrus Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,2018-04-01,"Single-player, Steam Achievements",1.99,False,773640,Nikita Ghostrus,2018
120441,Sacada,"['Casual', 'Indie', 'Strategy']",Logistical South Africa,2018-04-01,"Single-player, Steam Achievements, Steam Cloud...",4.99,False,733530,Sacada,2018
120442,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,2018-04-01,"Single-player, Steam Achievements, Steam Tradi...",1.99,False,610660,Laush Dmitriy Sergeevich,2018


In [72]:
df_steam_games.columns

Index(['Publisher', 'Genres', 'App_Name', 'Release_Date', 'Specs', 'Price',
       'Early_Access', 'Id', 'Developer', 'Release_Year'],
      dtype='object')

In [73]:
# New columns order
new_order = ['Id','App_Name','Price','Release_Year','Release_Date','Genres','Early_Access','Specs','Developer', 'Publisher']
df_steam_games= df_steam_games[new_order]

In [74]:
df_steam_games

,Id,App_Name,Price,Release_Year,Release_Date,Genres,Early_Access,Specs,Developer,Publisher
88310,761140,Lost Summoner Kitty,4.99,2018,2018-04-01,"['Action', 'Casual', 'Indie', 'Simulation',...",False,Single-player,Kotoshiro,Kotoshiro
88311,643980,Ironbound,0.00,2018,2018-04-01,"['Free to Play', 'Indie', 'RPG', 'Strategy']",False,"Single-player, Multi-player, Online Multi-Play...",Secret Level Srl,Making Fun Inc
88313,767400,,0.99,2017,2017-07-12,"['Action', 'Adventure', 'Casual']",False,Single-player,,
88315,772540,Battle Royale Trainer,3.99,2018,2018-04-01,"['Action', 'Adventure', 'Simulation']",False,"Single-player, Steam Achievements",Trickjump Games Ltd,Trickjump Games Ltd
88316,774276,Snow All Access Basic Pass,9.99,2018,2018-04-01,"['Free to Play', 'Indie', 'Simulation', 'Sp...",False,"Single-player, Multi-player, Online Multi-Play...",Poppermost Productions,Nan
...,...,...,...,...,...,...,...,...,...,...
120439,745400,Kebab It Up,1.99,2018,2018-04-01,"['Action', 'Adventure', 'Casual', 'Indie']",False,"Single-player, Steam Achievements, Steam Cloud",Bidoniera Games,Bidoniera Games
120440,773640,Colony On Mars,1.99,2018,2018-04-01,"['Casual', 'Indie', 'Simulation', 'Strategy']",False,"Single-player, Steam Achievements",Nikita Ghostrus,Ghostrus Games
120441,733530,Logistical South Africa,4.99,2018,2018-04-01,"['Casual', 'Indie', 'Strategy']",False,"Single-player, Steam Achievements, Steam Cloud...",Sacada,Sacada
120442,610660,Russian Roads,1.99,2018,2018-04-01,"['Indie', 'Racing', 'Simulation']",False,"Single-player, Steam Achievements, Steam Tradi...",Laush Dmitriy Sergeevich,Laush Studio


In [75]:
# Remove '[' and ']' characters as well as single quotes from the 'Genres' column
df_steam_games['Genres'] = df_steam_games['Genres'].str.replace(r"[\[\]']", '', regex=True)

In [76]:
df_steam_games

,Id,App_Name,Price,Release_Year,Release_Date,Genres,Early_Access,Specs,Developer,Publisher
88310,761140,Lost Summoner Kitty,4.99,2018,2018-04-01,"Action, Casual, Indie, Simulation, Strategy",False,Single-player,Kotoshiro,Kotoshiro
88311,643980,Ironbound,0.00,2018,2018-04-01,"Free to Play, Indie, RPG, Strategy",False,"Single-player, Multi-player, Online Multi-Play...",Secret Level Srl,Making Fun Inc
88313,767400,,0.99,2017,2017-07-12,"Action, Adventure, Casual",False,Single-player,,
88315,772540,Battle Royale Trainer,3.99,2018,2018-04-01,"Action, Adventure, Simulation",False,"Single-player, Steam Achievements",Trickjump Games Ltd,Trickjump Games Ltd
88316,774276,Snow All Access Basic Pass,9.99,2018,2018-04-01,"Free to Play, Indie, Simulation, Sports",False,"Single-player, Multi-player, Online Multi-Play...",Poppermost Productions,Nan
...,...,...,...,...,...,...,...,...,...,...
120439,745400,Kebab It Up,1.99,2018,2018-04-01,"Action, Adventure, Casual, Indie",False,"Single-player, Steam Achievements, Steam Cloud",Bidoniera Games,Bidoniera Games
120440,773640,Colony On Mars,1.99,2018,2018-04-01,"Casual, Indie, Simulation, Strategy",False,"Single-player, Steam Achievements",Nikita Ghostrus,Ghostrus Games
120441,733530,Logistical South Africa,4.99,2018,2018-04-01,"Casual, Indie, Strategy",False,"Single-player, Steam Achievements, Steam Cloud...",Sacada,Sacada
120442,610660,Russian Roads,1.99,2018,2018-04-01,"Indie, Racing, Simulation",False,"Single-player, Steam Achievements, Steam Tradi...",Laush Dmitriy Sergeevich,Laush Studio


In [77]:
# Save the DataFrame to a Parquet file with gzip compression
df_steam_games.to_parquet('GZIP/df_steam_games.gzip', compression='gzip')
# Save the DataFrame to a CSV file without including the index column
# df_steam_games.to_csv('CSV/steam_games.csv', index=False)

## australian_user_reviews.json etl

In [78]:
# Creating a list for the items
review = []
# Opening json
with open('datasets/australian_user_reviews.json', 'r', encoding='utf-8') as usersRevJson:
    for line in usersRevJson:
        # Append json items to the list
        review.append(ast.literal_eval(line))
# Close the json file explicitly
usersItemJson.close()
# Creating a dataframe
df_user_rev = pd.DataFrame(review)
# Viewing the dataframe
df_user_rev

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [79]:
# Explode data from the 'reviews' column and create new columns
allReviews = df_user_rev.explode('reviews') 
df_user_rev = pd.concat([allReviews.drop(['reviews'], axis=1), allReviews['reviews'].apply(pd.Series)], axis=1)
# View Dataframe
df_user_rev

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,0
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN
1,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN
...,...,...,...,...,...,...,...,...,...,...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,NaN
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,NaN
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,NaN
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D,NaN


In [80]:
# Remove the first newly created column and the 'user_url' column
df_user_rev = df_user_rev.drop([0, 'user_url'], axis=1)
df_user_rev

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
0,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
0,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
1,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
1,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...
25797,76561198312638244,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
25797,76561198312638244,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
25798,LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
25798,LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D


In [81]:
# Change the names of the columns
df_user_rev = df_user_rev.rename(columns={'user_id': 'User_Id', 'posted': 'Posted', 'item_id': 'Item_Id', 'helpful': 'Helpful', 'recommend': 'Recommend', 'review': 'Review'})
df_user_rev

,User_Id,funny,Posted,last_edited,Item_Id,Helpful,Recommend,Review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
0,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
0,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
1,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
1,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...
25797,76561198312638244,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
25797,76561198312638244,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
25798,LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
25798,LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D


In [82]:
# Counting NaN values in the 'User_Id' column
nan_count = df_user_rev['User_Id'].isna().sum()
print("Number of NaN values in User_Id column:", nan_count)

Number of NaN values in User_Id column: 0


In [83]:
# Count empty values in the 'funny' column
empty_count = df_user_rev['funny'].value_counts()['']
# Calculate the total percentage of empty values relative to the total number of rows
total_rows = len(df_user_rev)
empty_percentage = (empty_count / total_rows) * 100
# Print the number of empty values in the 'funny' column
print("Number of empty values in the 'funny' column:", empty_count)
# Print the total percentage of empty values in the 'funny' column
print("Total percentage of empty values in the 'funny' column: {:.2f}%".format(empty_percentage))

Number of empty values in the 'funny' column: 51154
Total percentage of empty values in the 'funny' column: 86.22%


In [84]:
# Count empty values in the 'last_edited' column
empty_count_last_edited = (df_user_rev['last_edited'] == '').sum()
# Calculate the total percentage of empty values in the 'last_edited' column
total_rows = len(df_user_rev)
empty_percentage_last_edited = (empty_count_last_edited / total_rows) * 100
# Print the number of empty values in the 'last_edited' column
print("Number of empty values in the 'last_edited' column:", empty_count_last_edited)
# Print the total percentage of empty values in the 'last_edited' column
print("Total percentage of empty values in the 'last_edited' column: {:.2f}%".format(empty_percentage_last_edited))

Number of empty values in the 'last_edited' column: 53165
Total percentage of empty values in the 'last_edited' column: 89.60%


In [85]:
# Remove the 'funny' and 'last_edited' columns due to having more than 80% NaN values
df_user_rev = df_user_rev.drop(columns=['funny', 'last_edited'])


In [86]:
# Count null 'posted'
null_in_Posted = df_user_rev['Posted'].isnull().sum()
print("Val Null 'posted':", null_in_Posted)

Val Null 'posted': 28


In [87]:
# Drop rows with null values in the 'Posted' column
df_user_rev.dropna(subset=['Posted'], inplace=True)

In [88]:
df_user_rev

,User_Id,Posted,Item_Id,Helpful,Recommend,Review
0,76561197970982479,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...
0,76561197970982479,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.
0,76561197970982479,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
1,js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
1,js41637,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...
25797,76561198312638244,Posted July 10.,70,No ratings yet,True,a must have classic from steam definitely wort...
25797,76561198312638244,Posted July 8.,362890,No ratings yet,True,this game is a perfect remake of the original ...
25798,LydiaMorley,Posted July 3.,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
25798,LydiaMorley,Posted July 20.,730,No ratings yet,True,:D


In [89]:
# Remove the text "Posted" from the dates in the 'Posted' column
df_user_rev['Posted'] = df_user_rev['Posted'].str.replace('Posted ', '')

In [90]:
df_user_rev['Posted']

0         November 5, 2011.
0            July 15, 2011.
0           April 21, 2011.
1            June 24, 2014.
1        September 8, 2013.
                ...        
25797              July 10.
25797               July 8.
25798               July 3.
25798              July 20.
25798               July 2.
Name: Posted, Length: 59305, dtype: object

In [91]:
# Format the data in the 'Posted' column to datetime, assigning NaT (Not a Time) to impossible formats
df_user_rev['Posted'] = pd.to_datetime(df_user_rev['Posted'], dayfirst=True, errors='coerce')


In [92]:
df_user_rev

,User_Id,Posted,Item_Id,Helpful,Recommend,Review
0,76561197970982479,2011-11-05,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
0,76561197970982479,2011-07-15,22200,No ratings yet,True,It's unique and worth a playthrough.
0,76561197970982479,2011-04-21,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
1,js41637,2014-06-24,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
1,js41637,2013-09-08,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...
25797,76561198312638244,NaT,70,No ratings yet,True,a must have classic from steam definitely wort...
25797,76561198312638244,NaT,362890,No ratings yet,True,this game is a perfect remake of the original ...
25798,LydiaMorley,NaT,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
25798,LydiaMorley,NaT,730,No ratings yet,True,:D


In [93]:
# Remove rows with NaT (Not a Time) values in the 'Posted' column
df_user_rev.dropna(subset=['Posted'], inplace=True)

In [94]:
# Check if the format is correct (YYYY-MM-DD)
df_user_rev['Posted'] = pd.to_datetime(df_user_rev['Posted']).dt.strftime('%Y-%m-%d')

In [95]:
df_user_rev

,User_Id,Posted,Item_Id,Helpful,Recommend,Review
0,76561197970982479,2011-11-05,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
0,76561197970982479,2011-07-15,22200,No ratings yet,True,It's unique and worth a playthrough.
0,76561197970982479,2011-04-21,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
1,js41637,2014-06-24,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
1,js41637,2013-09-08,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...
25764,wayfeng,2015-10-14,730,1 of 1 people (100%) found this review helpful,True,its FUNNNNNNNN
25765,76561198251004808,2015-10-10,253980,No ratings yet,True,Awesome fantasy game if you don't mind the gra...
25769,72947282842,2015-10-31,730,No ratings yet,True,Prettyy Mad Game
25771,ApxLGhost,2015-12-14,730,No ratings yet,True,AMAZING GAME 10/10


In [96]:
# Add a new column 'Year' containing the year extracted from the 'Posted' column
df_user_rev['Year'] = pd.to_datetime(df_user_rev['Posted']).dt.year

In [97]:
# Display the count of null values in each column of the DataFrame
null_values = df_user_rev.isnull().sum()
print(null_values)
df_user_rev

User_Id      0
Posted       0
Item_Id      0
Helpful      0
Recommend    0
Review       0
Year         0
dtype: int64


,User_Id,Posted,Item_Id,Helpful,Recommend,Review,Year
0,76561197970982479,2011-11-05,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011
0,76561197970982479,2011-07-15,22200,No ratings yet,True,It's unique and worth a playthrough.,2011
0,76561197970982479,2011-04-21,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2011
1,js41637,2014-06-24,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014
1,js41637,2013-09-08,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,2013
...,...,...,...,...,...,...,...
25764,wayfeng,2015-10-14,730,1 of 1 people (100%) found this review helpful,True,its FUNNNNNNNN,2015
25765,76561198251004808,2015-10-10,253980,No ratings yet,True,Awesome fantasy game if you don't mind the gra...,2015
25769,72947282842,2015-10-31,730,No ratings yet,True,Prettyy Mad Game,2015
25771,ApxLGhost,2015-12-14,730,No ratings yet,True,AMAZING GAME 10/10,2015


In [98]:
# Remove the 'Helpful' column from the DataFrame
df_user_rev = df_user_rev.drop(columns=['Helpful'])

In [99]:
df_user_rev

,User_Id,Posted,Item_Id,Recommend,Review,Year
0,76561197970982479,2011-11-05,1250,True,Simple yet with great replayability. In my opi...,2011
0,76561197970982479,2011-07-15,22200,True,It's unique and worth a playthrough.,2011
0,76561197970982479,2011-04-21,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011
1,js41637,2014-06-24,251610,True,I know what you think when you see this title ...,2014
1,js41637,2013-09-08,227300,True,For a simple (it's actually not all that simpl...,2013
...,...,...,...,...,...,...
25764,wayfeng,2015-10-14,730,True,its FUNNNNNNNN,2015
25765,76561198251004808,2015-10-10,253980,True,Awesome fantasy game if you don't mind the gra...,2015
25769,72947282842,2015-10-31,730,True,Prettyy Mad Game,2015
25771,ApxLGhost,2015-12-14,730,True,AMAZING GAME 10/10,2015


In [100]:
# Initialize the sentiment analyzer
sentiment_analyzer = SentimentIntensityAnalyzer()

# Function to assign sentiment scores according to the scale
def sentiment_score(text):
    if pd.isnull(text) or text == '':
        return 1  # Return neutral if it is empty or NaN
    elif isinstance(text, str):
        sentiment_score = sentiment_analyzer.polarity_scores(text)
        sentiment_score = sentiment_score['compound']
        if sentiment_score >= -0.05:
            return 2  # Good score
        elif sentiment_score <= -0.05:
            return 0  # Bad score
        else:
            return 1  # Neutral score
    else:
        return 1  # Return neutral for non-string values

# Convert the 'Review' column to string
df_user_rev['Review'] = df_user_rev['Review'].astype(str)
# Apply the function get_sentiment_score to the 'Review' column
df_user_rev['Sentiment_Score'] = df_user_rev['Review'].apply(sentiment_score)
# View the DataFrame
df_user_rev


,User_Id,Posted,Item_Id,Recommend,Review,Year,Sentiment_Score
0,76561197970982479,2011-11-05,1250,True,Simple yet with great replayability. In my opi...,2011,2
0,76561197970982479,2011-07-15,22200,True,It's unique and worth a playthrough.,2011,2
0,76561197970982479,2011-04-21,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011,2
1,js41637,2014-06-24,251610,True,I know what you think when you see this title ...,2014,2
1,js41637,2013-09-08,227300,True,For a simple (it's actually not all that simpl...,2013,2
...,...,...,...,...,...,...,...
25764,wayfeng,2015-10-14,730,True,its FUNNNNNNNN,2015,2
25765,76561198251004808,2015-10-10,253980,True,Awesome fantasy game if you don't mind the gra...,2015,2
25769,72947282842,2015-10-31,730,True,Prettyy Mad Game,2015,0
25771,ApxLGhost,2015-12-14,730,True,AMAZING GAME 10/10,2015,2


In [101]:
# Remove the 'Review' column from the DataFrame
df_user_rev = df_user_rev.drop(columns=['Review'])

In [102]:
df_user_rev

,User_Id,Posted,Item_Id,Recommend,Year,Sentiment_Score
0,76561197970982479,2011-11-05,1250,True,2011,2
0,76561197970982479,2011-07-15,22200,True,2011,2
0,76561197970982479,2011-04-21,43110,True,2011,2
1,js41637,2014-06-24,251610,True,2014,2
1,js41637,2013-09-08,227300,True,2013,2
...,...,...,...,...,...,...
25764,wayfeng,2015-10-14,730,True,2015,2
25765,76561198251004808,2015-10-10,253980,True,2015,2
25769,72947282842,2015-10-31,730,True,2015,0
25771,ApxLGhost,2015-12-14,730,True,2015,2


In [103]:
# Save the DataFrame to a CSV file named 'user_rev.csv' without including the index
# df_user_rev.to_csv('CSV/user_rev.csv', index=False)
# Also will save a PARQUET file too
df_user_rev.to_parquet('GZIP/df_user_revs.gzip', compression='gzip')


## Year Playtimes new Dataframe creation

In [104]:
# Import the first round of cleaned datasets to create new datasets
# Import the cleaned dataframe containing information about Steam games
df_steam_games = pd.read_parquet('GZIP/df_steam_games.gzip')
# Import the cleaned dataframe containing user item data
user_items = pd.read_parquet('GZIP/user_items.gzip')

In [105]:
# Get unique values of the 'Release_Year' column from the dataframe
unique_values_release_year = df_steam_games['Release_Year'].unique()
# Print the unique release years
print(unique_values_release_year)
# Print information about the 'Release_Year' column
print(df_steam_games['Release_Year'].info())

[2018 2017 1998 2006 2005 2002 2007 1995 1994 1993 2003 2008 1997 2009
 1999 1992 1989 2001 2000 2010 2011 2012 2013 2004 2014 1996 2016 1990
 2015 1988 1991 1987 1986 2019 1984 1985]
<class 'pandas.core.series.Series'>
Index: 10374 entries, 88310 to 120443
Series name: Release_Year
Non-Null Count  Dtype
--------------  -----
10374 non-null  int32
dtypes: int32(1)
memory usage: 121.6 KB
None


In [106]:
# Apply a lambda function to clean and split the 'Genres' column and assign the result to a new 'Genre' column
df_steam_games['Genre'] = df_steam_games['Genres'].apply(lambda x: x.strip("[]").replace("'", "").split(', '))
# Explode the 'Genre' column to create multiple rows for each game with different genres
df_steam_games = df_steam_games.explode('Genre')
# Display the updated dataframe
df_steam_games.head(3)

,Id,App_Name,Price,Release_Year,Release_Date,Genres,Early_Access,Specs,Developer,Publisher,Genre
88310,761140,Lost Summoner Kitty,4.99,2018,2018-04-01,"Action, Casual, Indie, Simulation, Strategy",False,Single-player,Kotoshiro,Kotoshiro,Action
88310,761140,Lost Summoner Kitty,4.99,2018,2018-04-01,"Action, Casual, Indie, Simulation, Strategy",False,Single-player,Kotoshiro,Kotoshiro,Casual
88310,761140,Lost Summoner Kitty,4.99,2018,2018-04-01,"Action, Casual, Indie, Simulation, Strategy",False,Single-player,Kotoshiro,Kotoshiro,Indie


In [107]:
# Remove the 'Genres' column from the dataframe
df_steam_games = df_steam_games.drop('Genres', axis=1)
# Display the updated dataframe
df_steam_games.head(2)

,Id,App_Name,Price,Release_Year,Release_Date,Early_Access,Specs,Developer,Publisher,Genre
88310,761140,Lost Summoner Kitty,4.99,2018,2018-04-01,False,Single-player,Kotoshiro,Kotoshiro,Action
88310,761140,Lost Summoner Kitty,4.99,2018,2018-04-01,False,Single-player,Kotoshiro,Kotoshiro,Casual


In [108]:
# Display information about the user_items dataframe
user_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5153209 entries, 0 to 88308
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Item_Id           object 
 1   Item_Name         object 
 2   Playtime_Forever  float64
 3   Playtime_2Weeks   float64
 4   User_Id           object 
 5   Steam_Id          object 
dtypes: float64(2), object(4)
memory usage: 275.2+ MB


In [109]:
# Filter user_items dataframe to remove rows where Playtime_Forever is equal to 0
user_items = user_items[user_items['Playtime_Forever'] != 0]

In [110]:
# Show information after deleting the Playtime_Forever that is equal to 0
user_items.head(2)

,Item_Id,Item_Name,Playtime_Forever,Playtime_2Weeks,User_Id,Steam_Id
0,10,Counter-Strike,6.0,0.0,76561197970982479,76561197970982479
0,30,Day of Defeat,7.0,0.0,76561197970982479,76561197970982479


In [111]:
# Calculate the total playtime per game by summing the Playtime_Forever column for each game
total_playtime_per_game = df_user_items.groupby(['Item_Id'])['Playtime_Forever'].sum()

In [112]:
# Display the first 2 entries of the total playtime per game
total_playtime_per_game.head(2)

Item_Id
10     17386015.0
100      311999.0
Name: Playtime_Forever, dtype: float64

In [113]:
# Merge the DataFrame df_steam_games with total_playtime_per_game based on the 'Id' column of df_steam_games and 'Item_Id' column of total_playtime_per_game, keeping only the matching records
playtime_releaseyear = df_steam_games.merge(total_playtime_per_game, left_on='Id', right_on='Item_Id', how='inner')

In [114]:
# Show df
playtime_releaseyear.head(2)

,Id,App_Name,Price,Release_Year,Release_Date,Early_Access,Specs,Developer,Publisher,Genre,Playtime_Forever
0,70,Halflife,9.99,1998,1998-08-11,False,"Single-player, Multi-player, Valve Anti-Cheat ...",Valve,Valve,Action,2682852.0
1,1640,Disciples Ii Galleans Return,6.99,2006,2006-06-07,False,"Single-player, Multi-player, Co-op",Strategy First,Strategy First,Strategy,27397.0


In [115]:
# Convert the 'Release_Year' column to integer type
playtime_releaseyear['Release_Year'] = playtime_releaseyear['Release_Year'].astype(int)
# Remove leading and trailing whitespace from the 'Genre' column
playtime_releaseyear['Genre'] = playtime_releaseyear['Genre'].str.strip()

In [116]:
# View unique Genres
print(playtime_releaseyear['Genre'].unique())

['Action' 'Strategy' 'Indie' 'RPG' 'Racing' 'Casual' 'Adventure' 'nan'
 'Simulation' 'Free to Play' 'Massively Multiplayer'
 'Animation &amp; Modeling' 'Sports' 'Early Access' 'Education'
 'Utilities' 'Web Publishing' 'Design &amp; Illustration'
 'Software Training' 'Video Production' 'Audio Production' 'Photo Editing']


In [117]:
# Show df
playtime_releaseyear.head(1)

,Id,App_Name,Price,Release_Year,Release_Date,Early_Access,Specs,Developer,Publisher,Genre,Playtime_Forever
0,70,Halflife,9.99,1998,1998-08-11,False,"Single-player, Multi-player, Valve Anti-Cheat ...",Valve,Valve,Action,2682852.0


In [118]:
# Group the dataframe by 'Genre' and 'Release_Year', then calculate the total playtime forever for each group
grouped = playtime_releaseyear.groupby(['Genre', 'Release_Year'])['Playtime_Forever'].sum().reset_index()

In [119]:
# View grouped df
grouped.head(3)

,Genre,Release_Year,Playtime_Forever
0,Action,1989,607.0
1,Action,1990,5.0
2,Action,1991,115.0


In [120]:
# Filter the grouped dataframe to retain only the rows with the maximum playtime forever for each genre
df_filtered_max_year_playtime = grouped.loc[grouped.groupby('Genre')['Playtime_Forever'].idxmax()]

In [121]:
# View filtered df
df_filtered_max_year_playtime.head(3)

,Genre,Release_Year,Playtime_Forever
24,Action,2013,184814548.0
56,Adventure,2013,102498364.0
65,Animation &amp; Modeling,2015,162422.0


In [122]:
# Save the hours_per_year in the CSV and GZIP folders
# df_user_rev.to_csv('CSV/hours_per_year.csv', index=False)
# Also will save a PARQUET file too
df_filtered_max_year_playtime.to_parquet('GZIP/max_hours_per_year.gzip', compression='gzip')

## UserForGenre

In [123]:
# Load the user items data from a compressed parquet file
user_items = pd.read_parquet('GZIP/user_items.gzip')
# Display the first row of the loaded user items dataframe
user_items.head(1)

,Item_Id,Item_Name,Playtime_Forever,Playtime_2Weeks,User_Id,Steam_Id
0,10,Counter-Strike,6.0,0.0,76561197970982479,76561197970982479


In [124]:
# Drop columns 'Item_Name', 'Playtime_2Weeks', and 'Steam_Id' from the user items dataframe
user_items_ids = user_items.drop(['Item_Name', 'Playtime_2Weeks', 'Steam_Id'], axis=1)
# Rename the column 'Item_Id' to 'Id' in the modified dataframe
user_items_ids = user_items_ids.rename(columns={'Item_Id': 'Id'})
# Display the first row of the modified dataframe
user_items_ids.head(1)

,Id,Playtime_Forever,User_Id
0,10,6.0,76561197970982479


In [125]:
# Display playtime df
playtime_releaseyear.head(1)

,Id,App_Name,Price,Release_Year,Release_Date,Early_Access,Specs,Developer,Publisher,Genre,Playtime_Forever
0,70,Halflife,9.99,1998,1998-08-11,False,"Single-player, Multi-player, Valve Anti-Cheat ...",Valve,Valve,Action,2682852.0


In [126]:
# Drop specified columns from the playtime_releaseyear dataframe
playtime_ids = playtime_releaseyear.drop(['App_Name', 'Price', 'Release_Year', 'Release_Date', 'Early_Access', 'Specs', 'Developer', 'Publisher', 'Playtime_Forever'], axis=1)
# Display the resulting dataframe
playtime_ids.head(2)

,Id,Genre
0,70,Action
1,1640,Strategy


In [127]:
# Save the playtime_ids dataframe to a Parquet file with gzip compression
playtime_ids.to_parquet('GZIP/playtime_ids.gzip', compression='gzip')
# Save the user_items_ids dataframe to a Parquet file with gzip compression
user_items_ids.to_parquet('GZIP/user_items_ids.gzip', compression='gzip')

In [128]:
# Read the user_items_ids dataframe from the Parquet file with gzip compression
user_items_ids = pd.read_parquet('GZIP/user_items_ids.gzip')
# Read the playtime_ids dataframe from the Parquet file with gzip compression
playtime_ids = pd.read_parquet('GZIP/playtime_ids.gzip')
# Display the first row of the playtime_ids dataframe
playtime_ids.head(1)

,Id,Genre
0,70,Action


In [129]:
# Merge the playtime_ids and user_items_ids dataframes based on the 'Id' column using a left join
user_playtime_items = playtime_ids.merge(user_items_ids, on="Id", how="left")

In [130]:
# Filter out rows where 'Playtime_Forever' is not equal to 0
user_playtime_items = user_playtime_items[user_playtime_items['Playtime_Forever'] != 0]
user_playtime_items_2 =  user_playtime_items.copy()

In [131]:
# Display DF
user_playtime_items

,Id,Genre,Playtime_Forever,User_Id
3,70,Action,108.0,doctr
5,70,Action,32.0,themanwich
6,70,Action,10.0,maplemage
7,70,Action,227.0,corrupted_soul
8,70,Action,36.0,cadmusthreepointoh
...,...,...,...,...
3528022,80,Action,203.0,76561198208507532
3528024,80,Action,244.0,76561198221578852
3528025,80,Action,7.0,EnVyIsSmexy
3528031,80,Action,9.0,76561198283312749


In [132]:
# Read the Parquet file 'df_user_revs.gzip' into a DataFrame
df_for_rev_times = pd.read_parquet('GZIP/df_user_revs.gzip')
# Display the first row of the DataFrame
df_for_rev_times.head(1)

,User_Id,Posted,Item_Id,Recommend,Year,Sentiment_Score
0,76561197970982479,2011-11-05,1250,True,2011,2


In [133]:
# Selecting specific columns 'User_Id', 'Item_Id', and 'Posted' from the DataFrame df_for_rev_times
df_for_rev_times = df_for_rev_times[['User_Id', 'Item_Id', 'Posted']]
# Displaying the first row of the resulting DataFrame
df_for_rev_times.head(1)

,User_Id,Item_Id,Posted
0,76561197970982479,1250,2011-11-05


In [134]:
# Show info about the df
df_for_rev_times.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49186 entries, 0 to 25780
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   User_Id  49186 non-null  object
 1   Item_Id  49186 non-null  object
 2   Posted   49186 non-null  object
dtypes: object(3)
memory usage: 1.5+ MB


In [135]:
# Convert 'Posted' column to datetime type
df_for_rev_times['Posted'] = pd.to_datetime(df_for_rev_times['Posted'])
# Create a new column 'Posted_Year' containing only the year
df_for_rev_times['Posted_Year'] = df_for_rev_times['Posted'].dt.year
# Display the DataFrame with the new column and DataFrame info
print(df_for_rev_times.info())
df_for_rev_times.head(1)


<class 'pandas.core.frame.DataFrame'>
Index: 49186 entries, 0 to 25780
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   User_Id      49186 non-null  object        
 1   Item_Id      49186 non-null  object        
 2   Posted       49186 non-null  datetime64[ns]
 3   Posted_Year  49186 non-null  int32         
dtypes: datetime64[ns](1), int32(1), object(2)
memory usage: 1.7+ MB
None


,User_Id,Item_Id,Posted,Posted_Year
0,76561197970982479,1250,2011-11-05,2011


In [136]:
# See if ther's any null in any column
nulls = df_for_rev_times.isnull().sum()
nulls

User_Id        0
Item_Id        0
Posted         0
Posted_Year    0
dtype: int64

In [137]:
# Export the GZIPS of the 2 DF's
df_for_rev_times.to_parquet('GZIP/df_for_rev_times.gzip', compression='gzip')
user_playtime_items.to_parquet('GZIP/user_playtime_items.gzip', compression='gzip')

In [138]:
# Reload DF's in case we already have them loaded
user_playtime_items = pd.read_parquet('GZIP/user_playtime_items.gzip')
print(user_playtime_items.info())
df_for_rev_times = pd.read_parquet('GZIP/df_for_rev_times.gzip')
print(df_for_rev_times.info())

<class 'pandas.core.frame.DataFrame'>
Index: 2370681 entries, 3 to 3528032
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Id                object 
 1   Genre             object 
 2   Playtime_Forever  float64
 3   User_Id           object 
dtypes: float64(1), object(3)
memory usage: 90.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 49186 entries, 0 to 25780
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   User_Id      49186 non-null  object        
 1   Item_Id      49186 non-null  object        
 2   Posted       49186 non-null  datetime64[ns]
 3   Posted_Year  49186 non-null  int32         
dtypes: datetime64[ns](1), int32(1), object(2)
memory usage: 1.7+ MB
None


In [139]:
#Display DF's
print(df_for_rev_times.head(1))
user_playtime_items.head(1)

             User_Id Item_Id     Posted  Posted_Year
0  76561197970982479    1250 2011-11-05         2011


,Id,Genre,Playtime_Forever,User_Id
3,70,Action,108.0,doctr


In [140]:
# Group the dataframe user_playtime_items by 'User_Id' and 'Genre',
# and sum the 'Playtime_Forever' for each group
grouped_users_hour_per_genre = user_playtime_items.groupby(['User_Id', 'Genre'])['Playtime_Forever'].sum()
# Reset the index of the resulting grouped dataframe to make it a regular DataFrame
grouped_users_hour_per_genre = grouped_users_hour_per_genre.reset_index()


In [141]:
# Show the needed data till now
user_playtime_items = user_playtime_items.rename(columns={'Id': 'Item_Id'})
user_playtime_items = user_playtime_items.drop(columns=['Playtime_Forever', 'Item_Id'])
df_for_rev_times = df_for_rev_times.drop(columns=['Posted'])
print(user_playtime_items.head(2))
print(df_for_rev_times.head(2)) # We need this one

    Genre     User_Id
3  Action       doctr
5  Action  themanwich
             User_Id Item_Id  Posted_Year
0  76561197970982479    1250         2011
0  76561197970982479   22200         2011


In [142]:
merge_posteds = pd.merge(df_for_rev_times, user_playtime_items, on='User_Id', how='left')

In [143]:
merge_posteds

,User_Id,Item_Id,Posted_Year,Genre
0,76561197970982479,1250,2011,Adventure
1,76561197970982479,1250,2011,Indie
2,76561197970982479,1250,2011,Strategy
3,76561197970982479,1250,2011,RPG
4,76561197970982479,1250,2011,Action
...,...,...,...,...
3018634,ApxLGhost,730,2015,Adventure
3018635,ApxLGhost,730,2015,Action
3018636,ApxLGhost,730,2015,Simulation
3018637,ApxLGhost,730,2015,Strategy


In [144]:
merge_posteds = merge_posteds.dropna(subset=['Genre'])

In [145]:
merge_posteds.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3013006 entries, 0 to 3018637
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   User_Id      object
 1   Item_Id      object
 2   Posted_Year  int32 
 3   Genre        object
dtypes: int32(1), object(3)
memory usage: 103.4+ MB


In [146]:
user_playtime_items_3 = user_playtime_items_2.drop(columns=['Id'])
user_playtime_items_3


,Genre,Playtime_Forever,User_Id
3,Action,108.0,doctr
5,Action,32.0,themanwich
6,Action,10.0,maplemage
7,Action,227.0,corrupted_soul
8,Action,36.0,cadmusthreepointoh
...,...,...,...
3528022,Action,203.0,76561198208507532
3528024,Action,244.0,76561198221578852
3528025,Action,7.0,EnVyIsSmexy
3528031,Action,9.0,76561198283312749


In [147]:
grouped_data = user_playtime_items_3.merge(merge_posteds, on=['User_Id', 'Genre'], how='left')

In [148]:
grouped_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108558013 entries, 0 to 108558012
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Genre             object 
 1   Playtime_Forever  float64
 2   User_Id           object 
 3   Item_Id           object 
 4   Posted_Year       float64
dtypes: float64(2), object(3)
memory usage: 4.0+ GB


In [149]:
grouped_data = grouped_data.drop_duplicates()

In [150]:
grouped_data

,Genre,Playtime_Forever,User_Id,Item_Id,Posted_Year
0,Action,108.0,doctr,250320,2013.0
92,Action,108.0,doctr,20920,2012.0
184,Action,108.0,doctr,204100,2012.0
276,Action,108.0,doctr,224600,2014.0
368,Action,108.0,doctr,207610,2012.0
...,...,...,...,...,...
108558000,Action,203.0,76561198208507532,303390,2015.0
108558009,Action,244.0,76561198221578852,NaN,NaN
108558010,Action,7.0,EnVyIsSmexy,NaN,NaN
108558011,Action,9.0,76561198283312749,NaN,NaN


In [151]:
grouped_data = grouped_data.dropna(subset=['Posted_Year'])
grouped_data.head(3)

,Genre,Playtime_Forever,User_Id,Item_Id,Posted_Year
0,Action,108.0,doctr,250320,2013.0
92,Action,108.0,doctr,20920,2012.0
184,Action,108.0,doctr,204100,2012.0


In [152]:
grouped_data_final = grouped_data.groupby('Genre').apply(lambda x: x[x['Playtime_Forever'] == x['Playtime_Forever'].max()])

In [153]:
grouped_data_final

Genre  Playtime_Forever  \
Genre                                                            
Action           101461114            Action          530882.0   
                 101461241            Action          530882.0   
                 101461368            Action          530882.0   
                 101461495            Action          530882.0   
                 101461622            Action          530882.0   
...                                      ...               ...   
Utilities        16777162          Utilities           65427.0   
Video Production 33270405   Video Production            4582.0   
Web Publishing   93576612     Web Publishing          102554.0   
                 93576613     Web Publishing          102554.0   
nan              101048655               nan           15820.0   

                                      User_Id Item_Id  Posted_Year  
Genre                                                               
Action           101461114        shinomegami    8500       2015.0  
                 101461241        shinomegami  417860       2015.0  
                 101461368        shinomegami    6910       2014.0  
                 101461495        shinomegami     550       2012.0  
                 101461622        shinomegami  416590       2015.0  
...                                       ...     ...          ...  
Utilities        16777162   76561198059330972  248170       2015.0  
Video Production 33270405   76561198090768760     440       2014.0  
Web Publishing   93576612   76561198035718256   45760       2014.0  
                 93576613   76561198035718256  222940       2015.0  
nan              101048655  76561197995146150  351555       2015.0  

[80 rows x 5 columns]

In [154]:
grouped_data_final.to_parquet('GZIP/max_playtime_user_genre.gzip', compression='gzip')

In [155]:
max_playtime_user_genre = pd.read_parquet('GZIP/max_playtime_user_genre.gzip')

In [156]:
max_playtime_user_genre

Genre  Playtime_Forever  \
Genre                                                            
Action           101461114            Action          530882.0   
                 101461241            Action          530882.0   
                 101461368            Action          530882.0   
                 101461495            Action          530882.0   
                 101461622            Action          530882.0   
...                                      ...               ...   
Utilities        16777162          Utilities           65427.0   
Video Production 33270405   Video Production            4582.0   
Web Publishing   93576612     Web Publishing          102554.0   
                 93576613     Web Publishing          102554.0   
nan              101048655               nan           15820.0   

                                      User_Id Item_Id  Posted_Year  
Genre                                                               
Action           101461114        shinomegami    8500       2015.0  
                 101461241        shinomegami  417860       2015.0  
                 101461368        shinomegami    6910       2014.0  
                 101461495        shinomegami     550       2012.0  
                 101461622        shinomegami  416590       2015.0  
...                                       ...     ...          ...  
Utilities        16777162   76561198059330972  248170       2015.0  
Video Production 33270405   76561198090768760     440       2014.0  
Web Publishing   93576612   76561198035718256   45760       2014.0  
                 93576613   76561198035718256  222940       2015.0  
nan              101048655  76561197995146150  351555       2015.0  

[80 rows x 5 columns]